In [ ]:
import pandas as pd
df1 = pd.read_csv("archive/aapl_2016_2020.csv")
df2 = pd.read_csv("archive/aapl_2021_2023.csv")

# Data Cleaning

In [ ]:
df = pd.concat([df1,df2],ignore_index=True)
del df1 
del df2
columns = df.columns
columns = [s.replace('[', '').replace(']', '').replace(' ', '').strip().lower() for s in columns]
df.columns = columns
df = df[["quote_date","underlying_last","expire_date","strike","dte",
    "c_iv","c_volume","c_last","c_bid","c_ask",
    "p_iv","p_volume","p_last","p_bid","p_ask",
    "strike_distance_pct","strike_distance"]]
df.sort_values('quote_date', inplace=True)

df.reset_index()
for column in df:
    if column == "quote_date" or column == "expire_date":
        continue
    df[column] = pd.to_numeric(df[column],errors="coerce")

df = df[df["c_volume"].notna()]
df = df[df["p_volume"].notna()]

# More data cleaning
Here we transform take the mean of the implied vol from puts and calls.  We also turn it into an odd dictionary (because I have a dictionary addiction) for actual processing.

In [ ]:
from tqdm import tqdm
keys = []
dates = {}
strikes = []
implied_vol = []
for i in tqdm(range(len(df))):
    key = df.iloc[i]["quote_date"]
    exp_date = df.iloc[i]["expire_date"]
    strike = df.iloc[i]["strike"]
    implied_vol = (df.iloc[i]["c_iv"] + df.iloc[i]["p_iv"])/2

    if key not in dates.keys():
        keys.append(key)
        
        dates[key] = [(exp_date,strike,implied_vol)]
        continue

    dates[key].append((exp_date,strike,implied_vol))
del df

# Interpellation and Graphing
Along with MORE DATA CLEANING (calculating time to expiry) we interpellate points onto a grid of predefined points.  We then plot it and save that figure.

In [ ]:
from datetime import datetime
import numpy as np
from scipy.interpolate import griddata
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
num_points = 20




for quote_time in tqdm(range(len(keys))):
    tte = []
    strikes = []
    vols = []
    triples = []
    date_str1 = keys[quote_time].strip()
    datetime1 = datetime.strptime(date_str1, "%Y-%m-%d")


    for k in range(len(dates[keys[quote_time]])):
        date_str2 = dates[keys[quote_time]][k][0].strip()
        datetime2 = datetime.strptime(date_str2, "%Y-%m-%d")
        difference =  datetime2- datetime1
        tte.append(int(difference.days))
        strikes.append(dates[keys[quote_time]][k][1])
        vols.append(dates[keys[quote_time]][k][2])
        triples.append((dates[keys[quote_time]][k][1],dates[keys[quote_time]][k][2],int(difference.days)))



    x_min, z_min = 50, 0
    x_max, z_max = 300, 365

    vol_linspace = np.linspace(0, 1,num_points) 
    strikes_linspace = np.linspace(x_min,x_max,num_points) 
    tte_linspace = np.linspace(z_min, z_max,num_points) 
    x = np.array(strikes)
    y = np.array(vols)
    z = np.array(tte)




    # Create ranges in each dimension for interpolation
    x_range = np.linspace(x_min, x_max, num=num_points)
    z_range = np.linspace(z_min, z_max, num=num_points)

    # Create a 2D grid of x and z
    grid_x, grid_z = np.meshgrid(x_range, z_range)

    # Interpolate y (values) over the grid
    grid_y = griddata((x, z), y, (grid_x, grid_z), method='nearest')

    # Convert the result to a pandas DataFrame for easier analysis
    grid_points = pd.DataFrame({
        'x': grid_x.ravel(),
        'z': grid_z.ravel(),
        'y': grid_y.ravel()  # These are the interpolated y values
    })

    grid_points = grid_points.fillna(grid_points.mean())



    # Create a 2D grid of values
    strike_prices = grid_points['x'].unique()
    time_to_expiry = grid_points['z'].unique()
    X, Y = np.meshgrid(strike_prices, time_to_expiry)
    X[X > x_max] = x_max
    X[X < x_min] = x_min
    

    Z = grid_points.set_index(['x', 'z'])['y'].unstack()
    Z[Z < 0] = 0
    Z[Z > 1] = 1
    Y[Y < z_min] = z_min
    Y[Y > z_max] = z_max

    # Create the 3D plot
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Plot the surface
    ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.7)

    # Set labels and title
    ax.set_xlim(x_min,x_max)
    ax.set_ylim(z_min,z_max)
    ax.set_zlim(0,1)
    ax.set_xlabel('Strike Price')
    ax.set_ylabel('Time to Expiry')
    ax.set_zlabel('Implied Volatility')
    ax.set_title('Surface Plot of Strike Price, Time to Expiry, and Implied Volatility')
    plt.savefig(f"figures/{quote_time}.png", dpi=100, bbox_inches='tight')

# Convert images into a gif

In [8]:
import imageio
from tqdm import tqdm
filenames = []
images = []
for quote_time in tqdm(range(len(keys))):
    filenames.append(f"figures/{quote_time}.png")
    #images.append(imageio.imread(f"figures/{quote_time}.png"))
print("Added all the images")
#imageio.mimsave('movie2.gif', images)
print("Done")
with imageio.get_writer('movie.gif', mode='I') as writer:
    for filename in tqdm(filenames):
        image = imageio.imread(filename)
        writer.append_data(image)
print("Done")

100%|██████████| 1823/1823 [00:00<00:00, 1992239.76it/s]


Added all the images
Done


  0%|          | 0/1823 [00:00<?, ?it/s]/var/folders/45/b8l6mxpx0qjbw3xwyc1866480000gn/T/ipykernel_85145/1453573889.py:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)
 55%|█████▍    | 997/1823 [00:05<00:04, 184.79it/s]